In [53]:
#access the bbc folder and assign a dictionary of {directory:files}

import os

source_path = '../bbc/'
mapping = {}
# looping through directory
for i in sorted(os.listdir(source_path)):
    # checking if it is directory or not
    if os.path.isdir(source_path+i):
        # creating the dictionary with class as key and first 300 files as key
        mapping[i] = sorted(os.listdir(source_path+i))[:50]
print(mapping.keys())

dict_keys(['business', 'entertainment', 'politics', 'sport', 'tech'])


In [54]:
# label or class or target list
label = []

# file name
file_name = []

# text file data list
data = []

# unpacking and iterating through dictionary
for i, j in mapping.items():
    # iterating through list of files for each class
    for k in j:
        # appending labels/class/target
        label.append(i)
        file_name.append(k)
        # reading the file and appending to data list
        data.append(open(source_path+i+"/"+k, encoding="cp1252").read().replace("\n", " "))
        


In [55]:
#connect to amazon comprehend

import boto3
from sagemaker import get_execution_role

client = boto3.client(service_name='comprehend', region_name='us-east-1')
role = get_execution_role()


In [59]:
#call the amazon detect_entities and store label and text

aws_ent_label = []
aws_ent_text = []
for txt_data in data:
    #Entities

    response = client.detect_entities(
        Text=txt_data[:5000],
        LanguageCode='en'
    )
    #print(response)
    aws_ent_label.append([x['Type'] for x in response['Entities']])
    aws_ent_text.append([x['Text'] for x in response['Entities']])



In [ ]:
#Let's compare spacy entity recognition with comprehend

#install spacy

!pip install spacy
!python -m spacy download en_core_web_sm


In [ ]:
#load spacy model

import en_core_web_sm

en_nlp = en_core_web_sm.load()


In [60]:
#create a spacy doc and access entities via doc.ents

spacy_ent_label = []
spacy_ent_text = []
for txt_data in data:
    doc = en_nlp(t xt_data[:5000])
    spacy_ent_label.append([ent.label_ for ent in doc.ents])
    spacy_ent_text.append([ent.text for ent in doc.ents])

                          


In [64]:
#store all the info in a dataframe

df_ents = pd.DataFrame()
df_ents["label"] = label
df_ents["filename"] = file_name
df_ents["aws_entity"] = aws_ent_label
df_ents["aws_text"] = aws_ent_text
df_ents["spacy_entity"] = spacy_ent_label
df_ents["spacy_text"] = spacy_ent_text



df_ents.head()


,label,filename,aws_entity,aws_text,spacy_entity,spacy_text
0,business,001.txt,"[ORGANIZATION, LOCATION, ORGANIZATION, QUANTIT...","[Time Warner, US, TimeWarner, 76%, $1.13bn, £6...","[ORG, GPE, ORG, PERCENT, MONEY, ORG, DATE, MON...","[Time Warner, US, TimeWarner, 76%, 1.13bn, Â£6..."
1,business,002.txt,"[PERSON, QUANTITY, ORGANIZATION, LOCATION, PER...","[Greenspan, almost three months, Federal Reser...","[PERSON, DATE, ORG, GPE, PERSON, GPE, GPE, MON...","[Greenspan, almost three months, the Federal R..."
2,business,003.txt,"[ORGANIZATION, OTHER, ORGANIZATION, QUANTITY, ...","[Yukos, Russian, Yukos, $900m, Â£479m, Rosneft...","[NORP, PRODUCT, MONEY, ORG, ORG, GPE, MONEY, G...","[Russian, Yukos, $900m, State, Rosneft, Yugans..."
3,business,004.txt,"[ORGANIZATION, ORGANIZATION, QUANTITY, QUANTIT...","[BA, British Airways, 40%, three months, 31 De...","[ORG, PERCENT, DATE, MONEY, GPE, DATE, PERSON,...","[BA, 40%, the three months to 31 December 2004..."
4,business,005.txt,"[ORGANIZATION, ORGANIZATION, LOCATION, ORGANIZ...","[Pernod, Domecq, UK, Allied Domecq, France, Pe...","[ORG, GPE, ORG, GPE, ORG, ORG, ORG, NORP, ORG,...","[Domecq , UK, Allied Domecq, France, Pernod R..."


In [100]:
aws_unique_text = []
spacy_unique_text = []
aws_unique_ent = []
spacy_unique_ent = []
for i in range(len(spacy_ent_text)):
    #store the entity content
    s1_text = set(spacy_ent_text[i])
    s2_text = set(aws_ent_text[i])

    #store the type of entity
    s1_ent_label = set(spacy_ent_label[i])
    s2_ent_label = set(aws_ent_label[i])
    
    if len(s1_text) < len(s2_text):
        aws_unique_text.append(set(s2_text-s1_text))
        spacy_unique_text.append("")
        aws_unique_ent.append(set(s2_ent_label - s1_ent_label))
        spacy_unique_ent.append("")


    

In [101]:
aws_unique_text = []
spacy_unique_text = []
aws_unique_ent = []
spacy_unique_ent = []
for i in range(len(spacy_ent_text)):
    #store the entity content
    s1_text = set(spacy_ent_text[i])
    s2_text = set(aws_ent_text[i])
    
    #store the type of entity
    s1_ent_label = set(spacy_ent_label[i])
    s2_ent_label = set(aws_ent_label[i])
    if len(s1_text) < len(s2_text):
        aws_unique_text.append(set(s2_text-s1_text))
        spacy_unique_text.append("")
        aws_uniqe_ent.append(set(s2_ent_label - s1_ent_label))
        spacy_unique_ent.append("")
    else:
        spacy_unique_text.append(set(s1_text-s2_text))
        aws_unique_text.append("")
        spacy_unique_ent.append(set(s1_ent_label - s2_ent_label))
        aws_unique_ent.append("")
        


NameError: name 'aws_uniqe_ent' is not defined

In [99]:
#create a diff between spacy and comprehend entities by finding unique entities

aws_unique_text = []
spacy_unique_text = []
aws_unique_label = []
spacy_unique_text = []
for i in range(len(spacy_ent_text)):
    #store the entity content
    s1_text = set(spacy_ent_text[i])
    s2_text = set(aws_ent_text[i])
    
    #store the type of entity
    s1_ent_label = set(spacy_ent_label[i])
    s2_ent_label = set(aws_ent_label[i])
    if len(s1_text) < len(s2_text):
        diff = set(s2_text - s1_text)
        aws_unique_text.append(diff)
        aws_unique_label.append([x['Type'] for x in diff])
        spacy_unique_text.append("")
        spacy_unique_label.append("")
    else:
        diff = set(s1_text - s2_text)
        spacy_unique_text.append(diff)
        spacy_unique_label.append(x.label_ for x in diff)
        aws_unique_text.append("")
        aws_unique_text.append("")
        
        
    
df = pd.DataFrame()

df["label"] = label
df["filename"] = file_name
df["spacy_unique_label"] = spacy_unique_label
df["spacy_unique_text"] = spacy_unique_text
df["aws_unique_label"] = aws_unique_label
df["aws_unique_text"] = aws_unique_text
df.head()


TypeError: string indices must be integers

In [96]:
#Find intersection
intersection_text = []
for i in range(len(spacy_ent_text)):
    #store the entity content
    s1_text = set(spacy_ent_text[i])
    s2_text = set(aws_ent_text[i])
    
    #store the type of entity
    s1_ent_label = set(spacy_ent_label[i])
    
    intersection_text.append(s1_text.intersection(s2_text))

df['intersection_text'] = intersection_text
df.head()

,label,filename,spacy_unique_text,aws_unique_text,intersection_text
0,business,001.txt,,"{$500m, fourth quarter profits, $3.36bn, £600m...","{third, TimeWarner, 27%, 6.4%, three quarters,..."
1,business,002.txt,"{The White House, a quarter, as many months, 1...",,"{this year, European, Chinese, Monday, almost ..."
2,business,003.txt,"{540, 9.3bn, 27.5bn, Yukos', State, Menatep Gr...",,"{Rosneft, Russia, Russian, Menatep Group, Yuga..."
3,business,004.txt,,"{£75m, £59m, 13,000 jobs, 31 December 2004, £6...","{BA, October, Dresdner Kleinwort Wasserstein, ..."
4,business,005.txt,,"{third, Dunkin' Donuts, Creek wine, 8.2bn euro...","{Scotland, 4%, 1.2%, Allied, London, two, Pari..."


In [97]:
df.to_csv("aws_spacy_comparison.csv")